# Python Reddit API Wrapper (PRAW) Data Pull

> Much of the following code is taken from GA DSI week05 PRAW_blankish.ipynb

In [1]:
import praw
import pandas as pd

#### Provide credentials so you can use the wrapper

In [2]:
#The info below is provided to the wrapper to pull from the API.

reddit = praw.Reddit(client_id = 'ZRBAsakarjxuoA',     #this is that personal use script key
                     client_secret = 'hKyTbEe9fYV9QnDsi1JekNaDB50', #this is that secret key
                     user_agent = 'HLV',    #Whatever name you gave your application
                     username = 'vandervalk',      #your Reddit user name
                     password = 'disraeligears111')      #your Reddit password

#### Create variable names for the subreddits you want to pull from

In [3]:
subreddit = reddit.subreddit('SubSimulatorGPT2')    #set to variable name, pick the name of a subreddit

In [4]:
altsubreddit = reddit.front

#### Specify what type of posts from those subreddits you want to pull

Each subreddit has five different ways of organizing the topics created by redditors: `.hot`, `.new`, `.controversial`, `.top`, and `.gilded`. You can also use `.search("SEARCH_KEYWORDS")` to get only results matching an engine search.

In [9]:
subreddit_new = subreddit.new(limit = 100)

In [10]:
subreddit_new_2 = altsubreddit.new(limit = 100)

#### Create a topics dictionary:

In [11]:
topics_dict = { "title":[],
                "score":[],
                "id":[],
                "url":[], 
                "comms_num": [],
                "created": [],
                "body":[]}

#Use a for loop to take the posts gathered by the wrapper and place them into a dictionary.

for item in subreddit_new:
    topics_dict["title"].append(item.title)
    topics_dict["score"].append(item.score)
    topics_dict["id"].append(item.id)
    topics_dict["url"].append(item.url)
    topics_dict["comms_num"].append(item.num_comments)
    topics_dict["created"].append(item.created)
    topics_dict["body"].append(item.selftext)

In [12]:
topics_dict2 = { "title":[],
                "score":[],
                "id":[],
                "url":[], 
                "comms_num": [],
                "created": [],
                "body":[]}

#Use a for loop to take the posts gathered by the wrapper and place them into a dictionary.

for item in subreddit_new_2:
    topics_dict2["title"].append(item.title)
    topics_dict2["score"].append(item.score)
    topics_dict2["id"].append(item.id)
    topics_dict2["url"].append(item.url)
    topics_dict2["comms_num"].append(item.num_comments)
    topics_dict2["created"].append(item.created)
    topics_dict2["body"].append(item.selftext)

In [13]:
sim_comments = []

for each_id in topics_dict['id']:
    submission = reddit.submission(each_id)
    comments = submission.comments
    for i in range(len(comments)):
        sim_comments.append(comments[i].body)

In [14]:
real_comments = []

for each_id in topics_dict2['id']:
    submission = reddit.submission(each_id)
    comments = submission.comments
    for i in range(len(comments)):
        real_comments.append(comments[i].body)

#### Convert the dictionaries into DataFrames. 

In [15]:
df = pd.DataFrame(sim_comments, columns=['comments'])

In [16]:
df.head()

,comments
0,I am really interested in this! Do you have t...
1,"Oh my goodness, this is the most awesome thing..."
2,I'm very interested in seeing this. It sounds ...
3,I've been looking into it but am having troubl...
4,Well... there's my first house...


#### Add a column to each dataframe keep track of which subreddit the data is from

In [17]:
df['real'] = 0

#### Repeat the process for the second subreddit

In [18]:
df2 = pd.DataFrame(real_comments, columns=['comments'])

In [19]:
df2.head()

,comments
0,Hello and welcome to r/LifeProTips!\n\nPlease ...
1,I'm thinking it'll get to the point where you ...
2,Gary Glitter's Rock and Roll Pt 2.
3,Changed My Mind by e-dubble
4,Gucci gang Gucci gang Gucci gang


#### Add that extra column

In [20]:
df2['real'] = 1

#### Concatenate the 2 dataframes together

In [21]:
finaldf = pd.concat([df, df2])

In [22]:
finaldf.head()

,comments,real
0,I am really interested in this! Do you have t...,0
1,"Oh my goodness, this is the most awesome thing...",0
2,I'm very interested in seeing this. It sounds ...,0
3,I've been looking into it but am having troubl...,0
4,Well... there's my first house...,0


In [23]:
finaldf.reset_index(inplace=True, drop=True)
finaldf.head()

,comments,real
0,I am really interested in this! Do you have t...,0
1,"Oh my goodness, this is the most awesome thing...",0
2,I'm very interested in seeing this. It sounds ...,0
3,I've been looking into it but am having troubl...,0
4,Well... there's my first house...,0


In [24]:
finaldf['real'].value_counts(normalize=True)

0    0.681954
1    0.318046
Name: real, dtype: float64

In [42]:
# finaldf.to_csv('./holdout_df.csv', index=False)